# Coleta de dados abertos da Câmara dos Deputados

**Objetivo**: Obter informações atualizadas sobre a(o)s deputada(o)s federais e suas respectivas despesas realizadas em 2022, por meio dos recursos públicos destinados à cota parlamentar - verba recebida pela(o)s deputada(o)s para custear o exercício de suas funções.  

**Fonte de dados**: <a href="https://dadosabertos.camara.leg.br/swagger/api.html">Dados Abertos da Câmara dos Deputados</a>

##Carga de pacotes

In [ ]:
# Carregamento dos pacotes necessários
import time
import json
import requests
import joblib
from google.colab import drive
from google.colab import auth
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload

##Credenciais de acesso

In [ ]:
# Autorizar acesso ao Google Drive
auth.authenticate_user()
drive_service = build('drive', 'v3')

In [ ]:
# Montar drive para armazenar conteúdo
drive.mount('/content/drive')

## Extração de dados da fonte

In [ ]:
# Captura dos parlamentares em exercício no ano de 2022 (parte da 56ª legislatura)
%%time
resp1 = requests.get("https://dadosabertos.camara.leg.br/api/v2/deputados?idLegislatura=56")
list_dept = resp1.json()['dados']
list_id = [e['id'] for e in list_dept]

In [ ]:
# Requisições das despesas dos deputados em 2022
%%time
dic_dept_desp = {}
for nid in list_id:
  listA = []
  for pg in range(1,100):
    resp2 = requests.get(f"https://dadosabertos.camara.leg.br/api/v2/deputados/{str(nid)}/despesas?idLegislatura=56&ano=2022&pagina={pg}&itens=100")
    try:
      listA += resp2.json()['dados']
    except:
      listA = listA
    time.sleep(0.5)
  dic_dept_desp.update({nid: listA})
  joblib.dump(dic_dept_desp[nid],f'drive/MyDrive/ObjetosPy/listdesp{nid}') # Salvamento parcial na pasta ObjetosPy criada previamente no Google Drive

In [ ]:
# Carregar dados parciais/totais coletados das despesas
dic_dept_desp = {}
for nid in list_id:
  try:
    listA = joblib.load(f'drive/MyDrive/ObjetosPy/listdesp{nid}')
    dic_dept_desp.update({nid: listA})
  except:
    pass

## Limpeza e transformação dos dados extraídos

In [ ]:
# Padronização dos e-mails dos deputados
for nid in list_id:
    for i, elem in enumerate(list_dept):
        if elem['email'] is None:
            list_dept[i]['email']='indisponibilizado'

In [ ]:
# Cópia dos dados de despesas extraídos da fonte
dic1_dept_desp = dic_dept_desp

In [ ]:
# Limpeza dos dados
for nid in list_id:
    for i, elem in enumerate(dic1_dept_desp[nid]):
        for char in "'.":
            dic1_dept_desp[nid][i]['nomeFornecedor']=elem['nomeFornecedor'].replace(char,"")
            dic1_dept_desp[nid][i]['tipoDespesa']=elem['tipoDespesa'].replace(char,"")

In [ ]:
# Cópia dos dados limpos
dic2_dept_desp = dic1_dept_desp

In [ ]:
# Transformação dos dados
dic_mes = {1:'Jan', 2:'Fev', 3:'Mar', 4:'Abr', 5:'Mai', 6:'Jun', 7:'Jul', 8:'Ago', 9:'Set', 10:'Out', 11:'Nov', 12:'Dez'}
for nid in list_id:
    for i, elem in enumerate(dic2_dept_desp[nid]):
        dic2_dept_desp[nid][i]['mes']=dic_mes[elem['mes']]

## Exportação dos dados pré-processados 

In [ ]:
# Exportação de dados selecionados dos deputados para arquivo JSON
list_dept_mod = []
for elem in list_dept:
    list_dept_mod.append({"id": elem['id'], "nome": elem['nome'], "email": elem['email'],\
                            "partido": elem['siglaPartido'], "uf": elem['siglaUf'], "foto": elem['urlFoto']})
with open('deputados2022.json', 'w', encoding='utf8') as arq:
    json.dump(list_dept_mod, arq, ensure_ascii=False)

In [ ]:
# Upload do arquivo JSON dos deputados para o Google Drive
file_metadata1 = {
  'name': 'deputados2022',
  'mimeType': 'application/json'
}
media1 = MediaFileUpload('deputados2022.json', 
                        mimetype='application/json',
                        resumable=True)
created1 = drive_service.files().create(body=file_metadata1,
                                       media_body=media1,
                                       fields='id').execute()

In [ ]:
# Atualização dos dados dos deputados federais
fileId1 = created1.get('id')
updated1 = drive_service.files().update(fileId=fileId1,
                                        body=file_metadata1,
                                        media_body=media1).execute()

In [ ]:
# Exportação de dados selecionados das despesas para arquivo JSON
list_dept_desp = []
for chv in dic2_dept_desp.keys():
    for elem in dic2_dept_desp[chv]:
        list_dept_desp.append({"id": chv, "tipo": elem['tipoDespesa'], "iddoc": elem['codDocumento'],\
                                "ano": elem['ano'], "mes": elem['mes'], "data": elem['dataDocumento'],\
                                "nomefornecedor": elem['nomeFornecedor'], "codfornecedor": elem['cnpjCpfFornecedor'],\
                                "valor": elem['valorLiquido']})
with open('despesas2022.json', 'w', encoding='utf8') as arq:
    json.dump(list_dept_desp, arq, ensure_ascii=False)

In [ ]:
# Upload do arquivo JSON das despesas para o Google Drive
file_metadata2 = {
  'name': 'despesas2022',
  'mimeType': 'application/json'
}
media2 = MediaFileUpload('despesas2022.json', 
                        mimetype='application/json',
                        resumable=True)
created2 = drive_service.files().create(body=file_metadata2,
                                       media_body=media2,
                                       fields='id').execute()

In [ ]:
# Atualização dos dados das despesas realizadas
fileId2 = created2.get('id')
updated2 = drive_service.files().update(fileId=fileId2,
                                        body=file_metadata2,
                                        media_body=media2).execute()